# GIẢI BÀI TOÁN CÁI BALO BẰNG GIẢI THUẬT THAM LAM 


**Cho tập các đồ vật có giá trị và trọng lượng như sau:**

$$( v_i , w_i) = \{ (2,7),(6,3),(8,3),(7,5),(3,4),(4,7),(6,5),
(5,4),(10,15),(9,10),(8,17),(11,3),(12,6),(15,11),(6,6),
(8,14),(13,4),(14,8), $$ <br/>
$$ (15,9),(16,10),(13,14),(14,17),(15,9),(26,24),(13,11),(9,17),(25,12),(26,14) \}$$ **với dung tích chứa tối đa của cái Balo là W = 30**

Trong bài toán Ba lô, mục tiêu của chúng ta là tìm ra sự kết hợp tối ưu của các đối tượng được ràng buộc bởi tổng trọng lượng W và đạt được lợi nhuận/giá trị cao nhất (v). 

## Import library và cho tập dữ liệu mẫu

In [88]:
import numpy as np
import time
v = np.array([2,6,8,7,3,4,6,5,10,9,8,11,12,15,6,8,13,14,15,16,13,14,15,26,13,9,25,26])
n = len(v)
w = np.array([7,3,3,5,4,7,5,4,15,10,17,3,6,11,6,14,4,8,9,10,14,17,9,24,11,17,12,14])
W = 30

## 1. Xếp ba lô dạng 0-1 
**The 0-1 knapsack problem**
- Hạn chế số đồ vật thuộc mỗi loại là 0 (không được chọn) và 1 (được chọn).
- Phát biểu bằng toán học:

$$ \mbox{Cực đại hóa: } \Sigma_{j=1}^n p_j x_j \quad \mbox{sao cho} \quad \Sigma_{j=1}^n w_j x_j \le c, \quad \quad x_j \in {\{0;1\}}, \quad j=1,\dots,n $$
 


In [104]:
# Hàm mục tiêu Tính điểm (v_i/w_i) và sắp xếp điểm giảm dần
def calculateScore01(v,w,n):
    score = np.array([])
    for i in range(n):
        score = np.append(score,round((v[i]/w[i]),3)) 
    # Sắp xếp chỉ số điểm theo thứ tự giảm dần
    scoreIndexs = np.argsort(-score)     
    return scoreIndexs

# Giải thuật tham lam Bài toán cái túi
def KnapSack01(v,w,n,W,shouldPrint= True):
    start_01 = time.time()
    sumItems = 0
    
    # Sắp xếp các mục theo điểm số 
    scoreIndexs =  calculateScore01(v,w,n) 
    itemsInKnapsack = np.zeros(n).astype(int)
    
    # Thêm đồ vật có điểm cao nhất, dưới dung tích chứa của Balo.
    for i in range(n):
        index = scoreIndexs[i]
        if sumItems + w[index] <= W :
            itemsInKnapsack[index] = 1 
            sumItems += w[index]
            if shouldPrint:
                print(f"Thêm ({v[index]}, {w[index]}) vào BaLo")   
    
    end_01 = time.time()
    # Chỉ số trả về của tập hợp mang lại lợi nhuận tối đa
    return itemsInKnapsack, {end_01 - start_01}

In [105]:
itemsInKnapsack, runTime = KnapSack01(v,w,n,W)

print("***************************")
print("Mảng:")
print(f"Giá trị: {v}")
print(f"Trọng lượng: {w}")
print(f"Danh sách đồ vật được chọn: {itemsInKnapsack}")
print("Giá trị (v) được chọn:    ",v[itemsInKnapsack==1])  
print("Trọng lượng (w) được chọn:",w[itemsInKnapsack==1])  
print("***************************")
print("Tổng giá trị:    " , np.sum(v[itemsInKnapsack==1]))
print("Tổng trọng lượng:" , np.sum(w[itemsInKnapsack==1]))
print("Thời gian thực hiện:" , runTime)

Thêm (11, 3) vào BaLo
Thêm (13, 4) vào BaLo
Thêm (8, 3) vào BaLo
Thêm (25, 12) vào BaLo
Thêm (6, 3) vào BaLo
Thêm (7, 5) vào BaLo
***************************
Mảng:
Giá trị: [ 2  6  8  7  3  4  6  5 10  9  8 11 12 15  6  8 13 14 15 16 13 14 15 26
 13  9 25 26]
Trọng lượng: [ 7  3  3  5  4  7  5  4 15 10 17  3  6 11  6 14  4  8  9 10 14 17  9 24
 11 17 12 14]
Danh sách đồ vật được chọn: [0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0]
Giá trị (v) được chọn:     [ 6  8  7 11 13 25]
Trọng lượng (w) được chọn: [ 3  3  5  3  4 12]
***************************
Tổng giá trị:     70
Tổng trọng lượng: 30
Thời gian thực hiện: {0.003995418548583984}


## 2. Xếp ba lô dạng phân số
Fractional Knapsack

In [95]:
def calculateScoreFrac(v,w,n):
    # Tính tỷ lệ giá trị trên khối lượng cho từng đồ vật
    ratios = [vi / wi for vi, wi in zip(v, w)]
    
    # Sắp xếp các đồ vật theo tỷ lệ giảm dần    
    items = sorted(zip(ratios, v, w), reverse=True)
    return items

def fractionalKnapsack(v,w,n,W, items = None):

    final_value = 0.0
    final_weight = 0.0
    selected_items = []
    if items is None:
        items = calculateScoreFrac(v, w, n)
    else:
        items
    for ratio, vi, wi in items:   
         
        # Nếu đồ vật có khối lượng nhỏ hơn hoặc bằng khối lượng còn lại của balo thì giảm khối lượng còn lại của balo
        # Cộng giá trị balo vào tổng giá trị cuối cùng, Thêm đồ vật vào danh sách các món hàng đã chọn  
        if wi <= W:
            W -= wi
            final_value += vi
            final_weight += wi
            selected_items.append((vi, wi))
        
        # Nếu không thể chứa món hàng vào túi hoàn toàn thì cộng giá trị đã thêm vào tổng giá trị cuối cùng,
        # Cộng khối lượng đã thêm vào tổng trọng lượng cuối cùng     
        else:
            added_weight = W
            added_value = vi * (W / wi)
            final_value += added_value
            final_weight += added_weight
            selected_items.append((added_value, added_weight))
            break

    # Trả về tổng giá trị, tổng trọng lượng và danh sách các món hàng đã chọn
    return final_value, final_weight, selected_items
 
total_value, total_weight, selected_items = fractionalKnapsack(v, w, n, W)
 
print("Tổng giá trị    :", total_value)
print("Tổng trọng lượng:", total_weight)
print("*****************")
for item in selected_items:
    print(f"Thêm ({item[0]}, {item[1]}) vào BaLo") 

Tổng giá trị    : 73.0
Tổng trọng lượng: 30.0
*****************
Thêm (11, 3) vào BaLo
Thêm (13, 4) vào BaLo
Thêm (8, 3) vào BaLo
Thêm (25, 12) vào BaLo
Thêm (12, 6) vào BaLo
Thêm (4.0, 2) vào BaLo


## 3. So sánh thời gian thực hiện với loại Sort khác nhau

### Merge Sort

In [87]:
def mergeSort(arr):
    if len(arr) <= 1:
        return arr
    
    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]
    
    left = mergeSort(left)
    right = mergeSort(right)
    
    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0
    
    while i < len(left) and j < len(right):
        if left[i][0] > right[j][0]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    
    while i < len(left):
        result.append(left[i])
        i += 1
    
    while j < len(right):
        result.append(right[j])
        j += 1
    
    return result

def calculateScore_Merge(v, w, n):
    # Tính tỷ lệ giá trị trên khối lượng cho từng đồ vật
    ratios = [vi / wi for vi, wi in zip(v, w)]
    
    # Kết hợp các đồ vật và tỷ lệ vào một danh sách
    items = list(zip(ratios, v, w))
    
    # Sắp xếp danh sách các đồ vật bằng merge sort
    sorted_items = mergeSort(items)
    
    return sorted_items


### Quick Sort

In [93]:
def partition(arr, low, high):
    pivot = arr[high][0]
    i = low - 1

    for j in range(low, high):
        if arr[j][0] >= pivot:
            i += 1
            arr[i], arr[j] = arr[j], arr[i]

    arr[i + 1], arr[high] = arr[high], arr[i + 1]
    return i + 1


def quickSort(arr, low, high):
    if low < high:
        pi = partition(arr, low, high)
        quickSort(arr, low, pi - 1)
        quickSort(arr, pi + 1, high)


def calculateScore_Quick(v, w, n):
    # Tính tỷ lệ giá trị trên khối lượng cho từng đồ vật
    ratios = [vi / wi for vi, wi in zip(v, w)]

    # Kết hợp các đồ vật và tỷ lệ vào một danh sách
    items = list(zip(ratios, v, w))

    # Sắp xếp danh sách các đồ vật bằng quick sort
    quickSort(items, 0, len(items) - 1)

    return items


### So sánh 

In [94]:
# Hàm fractionalKnapsack với cách sắp xếp merge sort
start_time = time.time()
total_value, total_weight, selected_items = fractionalKnapsack(v, w, n, W, items = None)
end_time = time.time()
merge_sort_time = end_time - start_time

# Hàm fractionalKnapsack với cách sắp xếp quick sort
start_time = time.time()
items = calculateScore_Quick(v, w, n)
total_value, total_weight, selected_items = fractionalKnapsack(v, w, n, W, items)
end_time = time.time()
quick_sort_time = end_time - start_time

# Hàm fractionalKnapsack với cách sắp xếp sorted
start_time = time.time()
items = calculateScoreFrac(v, w, n)
total_value, total_weight, selected_items = fractionalKnapsack(v, w, n, W, items)
end_time = time.time()
sorted_time = end_time - start_time

# In kết quả
print("Thời gian thực hiện với merge sort:", merge_sort_time)
print("Thời gian thực hiện với quick sort:", quick_sort_time)
print("Thời gian thực hiện với sorted:", sorted_time)



Thời gian thực hiện với merge sort: 0.0003249645233154297
Thời gian thực hiện với quick sort: 0.0002536773681640625
Thời gian thực hiện với sorted: 0.00017333030700683594
